## Загрузим нужные библиотеки

In [132]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import numpy as np
from sklearn.metrics import f1_score
# нужные библиотеки
import numpy as np
import pandas as pd
import gc
import random
import re
import typing
import numpy as np
import datetime
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
from sklearn.metrics import roc_auc_score
import numpy as np
import optuna
import catboost as cb
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

import pandas as pd
from sklearn.metrics import f1_score

from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task


# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
%matplotlib inline

In [32]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)



In [11]:
df_salaries = pd.read_csv('Медианная_зарплата.csv',header = None,names = ['Год','Регион','Сумма'])
df_salaries = df_salaries[df_salaries['Год']==2017].drop(columns = ['Год'])

In [12]:
import difflib 

def fuzzy_merge(df1, df2, left_on, right_on, how='left', cutoff=0.95):
    df_other= df2.copy()
    df_other[left_on] = [get_closest_match(x, df1[left_on], cutoff) 
                         for x in df_other[right_on]]
    return df1.merge(df_other, on=left_on, how=how).drop_duplicates()

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, cutoff=cutoff)
    return matches[0] if matches else None


In [133]:
def year_of_birth_transform(df):
    dataframe = df.copy()
    dataframe['Год_Рождения'] = pd.to_datetime(df['Дата_Рождения']).dt.year
    dataframe.drop(columns=  ['Дата_Рождения'],inplace= True)
    return dataframe

def age_of_admission_transform(df):
    dataframe = df.copy()
    dataframe['Возраст_поступления'] = 1.0*(df['Год_Поступления'] - df['Год_Рождения'])
    dataframe['Возраст_поступления'].fillna(-1.0)
    dataframe['Перерыв_обучения'] = dataframe['Возраст_поступления'] - dataframe['Возраст_выпуска_УЗ']

    return dataframe

def age_of_graduation_transform(df):
    dataframe = df.copy()
    dataframe['Возраст_выпуска_УЗ'] = df['Год_Окончания_УЗ'] - df['Год_Рождения']
    dataframe['Возраст_выпуска_УЗ'].fillna(-1.0)
    return dataframe

def full_family_transform(df):
    dataframe = df.copy()
    dataframe['Мигранты'] = np.array(1*(df['Страна_ПП']!=df['Страна_Родители']))
    dataframe['Полная_семья'] = np.array(1*(df['Наличие_Матери']*df['Наличие_Отца']),dtype = int)
    return dataframe

def organization_type_transform(df):
    dataframe = df.copy()
    dataframe['Спецшкола'] = 1*np.array([('лицей' in str(x).lower()) or ('гимназия' in str(x).lower()) or ('сунц' in str(x).lower()) or ('одар' in str(x).lower()) for x in df['Уч_Заведение'].values])
    dataframe['Номерная_школа'] = 1*np.array([('№' in str(x).lower()) for x in df['Уч_Заведение']])
    dataframe['Городская школа'] = 1*np.array([(' г' in str(x).lower()) or (' г.' in str(x).lower()) for x in df['Где_Находится_УЗ']])
    dataframe['Учеба_Домашний_Регион'] = 1*np.array([(str(x['Регион_ПП']).lower() in str(x['Где_Находится_УЗ']).lower()) for i,x in dataframe.iterrows()])
    dataframe['Высшее_образование'] = 1*np.array([('универ' in str(s).lower()) or ('академия' in str(s).lower()) or ('впо' in str(s).lower()) or ('высшая школа' in str(s).lower()) or ('институт' in str(s).lower()) for s in df['Уч_Заведение'].values])
    dataframe['Среднее_образование'] = 1*np.array([('СОШ' in str(s)) or ('лицей' in str(s).lower()) or  ('редняя школа' in str(s).lower()) or ('гимназия' in str(s).lower()) or ('техникум' in str(s).lower()) or ('колледж' in str(s).lower()) or ('средняя' in str(s).lower()) for s in df['Уч_Заведение'].values])
    return dataframe


def lang_transform(d):
    df = d.copy()
    df['Изучал_Английский'] = 1*['англ' in str(s).lower() for s in df['Изучаемый_Язык'].values]
    df['Изучал_Немецкий'] = 1*['немец' in str(s).lower() for s in df['Изучаемый_Язык'].values]
    df['Изучал_Французский'] = 1*['франц' in str(s).lower() for s in df['Изучаемый_Язык'].values]
    df['Изучал_Русский'] = 1*['русск' in str(s).lower() for s in df['Изучаемый_Язык'].values]
    df['Изучал_Китайский'] = 1*['китай' in str(s).lower() for s in df['Изучаемый_Язык'].values]
    df['Количество_языков'] = df['Изучал_Английский'].astype(int) + df['Изучал_Немецкий'].astype(int) + df['Изучал_Французский'].astype(int) + df['Изучал_Русский'].astype(int) + df['Изучал_Китайский'].astype(int)
    return df

def normalize_grades(d):
    df = d.copy()
    df['НормБалл'] = df['СрБаллАттестата'].fillna(0.7)
    df['НормБалл'][df['СрБаллАттестата']<=5.001]/=5.0
    df['НормБалл'][(df['СрБаллАттестата']>=5.01) & (df['СрБаллАттестата']<=10.001)]/=10.0
    df['НормБалл'][(df['СрБаллАттестата']>=10.01) & (df['СрБаллАттестата']<=100.001)]/=100.0
    df['НормБалл'][(df['СрБаллАттестата']>=100.01) & (df['СрБаллАттестата']<=1000.001)]/=1000.0
    df['НормБалл'][(df['СрБаллАттестата']>=1000.01) & (df['СрБаллАттестата']<=5000.001)]/=5000.0
    df['ЛогНормБалл'] = 10*np.log10(10*df['НормБалл'].fillna(-1))
    return df

def add_salary_residence_region(d):
    df = d.copy()
    df['Регион_ПП'] = df['Регион_ПП'].fillna('Неизвестно')
    df = fuzzy_merge(df, df_salaries, left_on='Регион_ПП', right_on='Регион').drop(columns = 'Регион',inplace = False)
    df['Лог_Медиан_Зарплата'] = np.log(np.abs(df['Сумма'].fillna(0).values +1.0))
    df['Сумма'].fillna(-1.0)
    return df

def make_grouped_features(tr_data, te_data=None, test_flag=False):
    print('Начал генерировать признаки')
    columns_of_interest = ['Уч_Заведение','Год_Окончания_УЗ','Основания','Год_Поступления']
    if test_flag: # Если тестовая выборка, то пришиваем еще историю из тренировочной выборки
        data = pd.concat([tr_data, te_data], ignore_index = True)
        print('Использую трейн+тест')
    else:
        print('Использую трейн')
        data = tr_data.copy() # Если выборка тренировчная, то ничего не делаем, чтобы не заглянуть в будущее
    
    
    data['Уч_Заведение_Балл_mean'] = data[['Уч_Заведение','НормБалл']].fillna(0.7).groupby('Уч_Заведение', as_index=False).transform('mean')
    data['Уч_Заведение_Балл_median'] = data[['Уч_Заведение','НормБалл']].fillna(0.7).groupby('Уч_Заведение', as_index=False).transform('median')
    data['Уч_Заведение_Балл_std'] = data[['Уч_Заведение','НормБалл']].fillna(0.7).groupby('Уч_Заведение', as_index=False).transform('std')
    data['Год_Окончания_Уз_Балл_std'] = data[['Год_Окончания_УЗ','НормБалл']].fillna(0.7).groupby('Год_Окончания_УЗ', as_index=False).transform('std')
    data['Год_Окончания_Уз_Балл_mean'] = data[['Год_Окончания_УЗ','НормБалл']].fillna(0.7).groupby('Год_Окончания_УЗ', as_index=False).transform('mean')
    data['Год_Окончания_Уз_Балл_median'] = data[['Год_Окончания_УЗ','НормБалл']].fillna(0.7).groupby('Год_Окончания_УЗ', as_index=False).transform('median')
    data['Основания_Балл_std'] = data[['Основания','НормБалл']].fillna(0.7).groupby('Основания', as_index=False).transform('std')
    data['Основания_Балл_mean'] = data[['Основания','НормБалл']].fillna(0.7).groupby('Основания', as_index=False).transform('mean')
    data['Основания_Балл_median'] = data[['Основания','НормБалл']].fillna(0.7).groupby('Основания', as_index=False).transform('median')
    data['Год_Поступления_Балл_std'] = data[['Год_Поступления','НормБалл']].fillna(.7).groupby('Год_Поступления', as_index=False).transform('std')
    data['Год_Поступления_Балл_mean'] = data[['Год_Поступления','НормБалл']].fillna(.7).groupby('Год_Поступления', as_index=False).transform('mean')
    data['Год_Поступления_Балл_median'] = data[['Год_Поступления','НормБалл']].fillna(.7).groupby('Год_Поступления', as_index=False).transform('median')
    
    if test_flag: # Есть это тест, возвращаем только тестовую часть
        data = data.iloc[len(tr_data):]
        assert len(data)==len(te_data)
        return data
    print('Новые признаки готовы!')
    return data

In [134]:
df = pd.read_csv("./train.csv")
df = year_of_birth_transform(df)
df = age_of_graduation_transform(df)
df = age_of_admission_transform(df)
df = full_family_transform(df)
df = lang_transform(df)
df = organization_type_transform(df)
df = normalize_grades(df)
#df = make_grouped_features(df)
#df = add_salary_residence_region(df)

/home/kglin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kglin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kglin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kglin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: Set

In [135]:
N_THREADS = 8 # threads cnt for lgbm and linear models
N_FOLDS = 10 # folds cnt for AutoML
RANDOM_STATE = 32 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 12000 # Time in seconds for automl run
TARGET_NAME = 'Статус'

In [136]:
task = Task('multiclass', metric = lambda y_true, y_pred: f1_score(y_true, np.argmax(y_pred,axis =-1),average = 'macro'))

In [137]:
roles = {
    'target': TARGET_NAME,
    'drop': ['ID'],
}

In [138]:
automl = TabularUtilizedAutoML(task = task, 
                               timeout = TIMEOUT,
                               cpu_limit = N_THREADS,
                               reader_params = {'n_jobs': N_THREADS,'cv':N_FOLDS,'random_state':RANDOM_STATE})


In [139]:
automl.fit_predict(df, roles = roles,verbose = 3)


[06:23:02] Start automl utilizator with listed constraints:
[06:23:02] - time: 12000.00 seconds
[06:23:02] - CPU: 8 cores
[06:23:02] - memory: 16 GB

[06:23:02] If one preset completes earlier, next preset configuration will be started

[06:23:02] ==================================================
[06:23:02] Start 0 automl preset configuration:
[06:23:02] conf_0_sel_type_0.yml, random state: {'reader_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
[06:23:02] Found reader_params in kwargs, need to combine
[06:23:02] Merged variant for reader_params = {'n_jobs': 8, 'cv': 10, 'random_state': 42}
[06:23:02] Stdout logging level is INFO3.
[06:23:02] Task: multiclass

[06:23:02] Start automl preset with listed constraints:
[06:23:02] - time: 12000.00 seconds
[06:23:02] - CPU: 8 cores
[06:23:02] - memory: 16 GB

[06:23:02] Train data shape: (13584, 43)

[06:23:04] Feats was rejected during automatic roles guess: []
[06:23:04] Layer 1 train process start. Ti

[06:23:28] Linear model: C = 5e-05 score = 0.5677045263813314
[06:23:29] Linear model: C = 0.0001 score = 0.5781450317499318
[06:23:29] Linear model: C = 0.0005 score = 0.6479794175052559
[06:23:29] Linear model: C = 0.001 score = 0.6799735765370256
[06:23:29] Linear model: C = 0.005 score = 0.7481113868210643
[06:23:30] Linear model: C = 0.01 score = 0.7580170332970875
[06:23:30] Linear model: C = 0.05 score = 0.758461246521584
[06:23:30] Linear model: C = 0.1 score = 0.7613857606413438
[06:23:31] Linear model: C = 0.5 score = 0.7619396584205655
[06:23:31] Linear model: C = 1 score = 0.7619396584205655
[06:23:31] Linear model: C = 5 score = 0.7619396584205655
[06:23:31] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.7563513598737996
[06:23:31] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[06:23:31] Time left 11970.81 secs

[06:23:32] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[06:23:32] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_0_LightGBM =====
[

[06:25:04] [200]	valid's multi_logloss: 0.272915	valid's Opt metric: 0.726088
[06:25:06] [300]	valid's multi_logloss: 0.260458	valid's Opt metric: 0.739257
[06:25:08] [400]	valid's multi_logloss: 0.257099	valid's Opt metric: 0.733361
[06:25:09] Trial 2 with hyperparameters {'feature_fraction': 0.5290418060840998, 'num_leaves': 223, 'bagging_fraction': 0.8005575058716043, 'min_sum_hessian_in_leaf': 0.679657809075816, 'reg_alpha': 1.5320059381854043e-08, 'reg_lambda': 5.360294728728285} scored 0.7408986115636872 in 0:00:08.720573
[06:25:09] [1]	valid's multi_logloss: 0.793916	valid's Opt metric: 0.251641
[06:25:09] Training until validation scores don't improve for 200 rounds
[06:25:11] [100]	valid's multi_logloss: 0.313974	valid's Opt metric: 0.72565
[06:25:13] [200]	valid's multi_logloss: 0.26849	valid's Opt metric: 0.740387
[06:25:15] [300]	valid's multi_logloss: 0.26157	valid's Opt metric: 0.742073
[06:25:17] [400]	valid's multi_logloss: 0.264642	valid's Opt metric: 0.734112
[06:25:1

[06:26:31] Training until validation scores don't improve for 200 rounds
[06:26:31] [100]	valid's multi_logloss: 0.32984	valid's Opt metric: 0.707572
[06:26:32] [200]	valid's multi_logloss: 0.275964	valid's Opt metric: 0.732974
[06:26:33] [300]	valid's multi_logloss: 0.263474	valid's Opt metric: 0.736123
[06:26:34] [400]	valid's multi_logloss: 0.2598	valid's Opt metric: 0.747921
[06:26:34] [500]	valid's multi_logloss: 0.258492	valid's Opt metric: 0.759195
[06:26:35] [600]	valid's multi_logloss: 0.257966	valid's Opt metric: 0.758601
[06:26:36] [700]	valid's multi_logloss: 0.256738	valid's Opt metric: 0.764487
[06:26:36] [800]	valid's multi_logloss: 0.256382	valid's Opt metric: 0.761058
[06:26:37] [900]	valid's multi_logloss: 0.25602	valid's Opt metric: 0.762949
[06:26:37] Trial 12 with hyperparameters {'feature_fraction': 0.6104881284818561, 'num_leaves': 18, 'bagging_fraction': 0.9559213589718202, 'min_sum_hessian_in_leaf': 0.03299263281612611, 'reg_alpha': 0.005849052891051126, 'reg_l

[06:28:01] [100]	valid's multi_logloss: 0.313232	valid's Opt metric: 0.742511
[06:28:04] [200]	valid's multi_logloss: 0.266905	valid's Opt metric: 0.754389
[06:28:07] [300]	valid's multi_logloss: 0.265402	valid's Opt metric: 0.757566
[06:28:10] [400]	valid's multi_logloss: 0.272736	valid's Opt metric: 0.755938
[06:28:11] Trial 22 with hyperparameters {'feature_fraction': 0.8666972847526295, 'num_leaves': 128, 'bagging_fraction': 0.8877573202942047, 'min_sum_hessian_in_leaf': 0.0013892993485431736, 'reg_alpha': 0.13875598475241455, 'reg_lambda': 9.091952830189061e-05} scored 0.7517290827490809 in 0:00:12.604649
[06:28:12] [1]	valid's multi_logloss: 0.793383	valid's Opt metric: 0.251641
[06:28:12] Training until validation scores don't improve for 200 rounds
[06:28:14] [100]	valid's multi_logloss: 0.312232	valid's Opt metric: 0.742523
[06:28:16] [200]	valid's multi_logloss: 0.266712	valid's Opt metric: 0.743433
[06:28:19] [300]	valid's multi_logloss: 0.264764	valid's Opt metric: 0.74681


...

[09:31:03] 399:	learn: 0.2403136	test: 0.2915815	best: 0.2915786 (396)	total: 1.04s	remaining: 6.76s
[09:31:03] 499:	learn: 0.2249312	test: 0.2857967	best: 0.2857758 (498)	total: 1.29s	remaining: 6.45s
[09:31:04] 599:	learn: 0.2124592	test: 0.2829769	best: 0.2829730 (598)	total: 1.59s	remaining: 6.35s
[09:31:04] 699:	learn: 0.2009359	test: 0.2812769	best: 0.2812769 (699)	total: 1.85s	remaining: 6.09s
[09:31:04] 799:	learn: 0.1906203	test: 0.2799996	best: 0.2799581 (776)	total: 2.11s	remaining: 5.79s
[09:31:04] 899:	learn: 0.1815618	test: 0.2781784	best: 0.2781784 (899)	total: 2.38s	remaining: 5.55s
[09:31:05] 999:	learn: 0.1722231	test: 0.2785179	best: 0.2781784 (899)	total: 2.63s	remaining: 5.26s
[09:31:05] Stopped by overfitting detector  (100 iterations wait)
[09:31:05] bestTest = 0.278178401
[09:31:05] bestIteration = 899
[09:31:05] Shrink model to first 900 iterations.
[09:31:05] Trial 64 with hyperparameters {'max_depth': 5, 'nan_mode': 'Min', 'l2_leaf_reg': 0.0010239917810052597

[09:31:22] 199:	learn: 0.2769600	test: 0.3046204	best: 0.3046204 (199)	total: 614ms	remaining: 8.59s
[09:31:22] 299:	learn: 0.2579400	test: 0.2982016	best: 0.2982016 (299)	total: 927ms	remaining: 8.34s
[09:31:23] 399:	learn: 0.2404834	test: 0.2913608	best: 0.2913608 (399)	total: 1.23s	remaining: 8s
[09:31:23] 499:	learn: 0.2253571	test: 0.2857585	best: 0.2857401 (498)	total: 1.54s	remaining: 7.68s
[09:31:23] 599:	learn: 0.2127109	test: 0.2825299	best: 0.2825299 (599)	total: 1.84s	remaining: 7.38s
[09:31:23] 699:	learn: 0.2017882	test: 0.2810416	best: 0.2810416 (699)	total: 2.16s	remaining: 7.11s
[09:31:24] 799:	learn: 0.1919653	test: 0.2792107	best: 0.2792107 (799)	total: 2.47s	remaining: 6.8s
[09:31:24] 899:	learn: 0.1829319	test: 0.2776006	best: 0.2775827 (898)	total: 2.78s	remaining: 6.5s
[09:31:24] Stopped by overfitting detector  (100 iterations wait)
[09:31:24] bestTest = 0.2775827265
[09:31:24] bestIteration = 898
[09:31:24] Shrink model to first 899 iterations.
[09:31:24] Trial

[09:31:41] 199:	learn: 0.2804881	test: 0.2907564	best: 0.2907564 (199)	total: 525ms	remaining: 7.35s
[09:31:42] 299:	learn: 0.2606864	test: 0.2808110	best: 0.2808110 (299)	total: 782ms	remaining: 7.04s
[09:31:42] 399:	learn: 0.2429332	test: 0.2717272	best: 0.2717272 (399)	total: 1.06s	remaining: 6.88s
[09:31:42] 499:	learn: 0.2290339	test: 0.2673223	best: 0.2671852 (481)	total: 1.32s	remaining: 6.6s
[09:31:43] 599:	learn: 0.2166865	test: 0.2643810	best: 0.2643810 (599)	total: 1.57s	remaining: 6.29s
[09:31:43] 699:	learn: 0.2056551	test: 0.2628050	best: 0.2625806 (685)	total: 1.82s	remaining: 5.99s
[09:31:43] 799:	learn: 0.1950965	test: 0.2605338	best: 0.2605338 (799)	total: 2.09s	remaining: 5.75s
[09:31:43] 899:	learn: 0.1858747	test: 0.2592646	best: 0.2592330 (898)	total: 2.34s	remaining: 5.46s
[09:31:44] 999:	learn: 0.1771324	test: 0.2578932	best: 0.2577090 (990)	total: 2.61s	remaining: 5.22s
[09:31:44] 1099:	learn: 0.1690236	test: 0.2578468	best: 0.2570450 (1068)	total: 2.88s	remain

[09:32:01] 999:	learn: 0.1783701	test: 0.2623973	best: 0.2623710 (996)	total: 2.86s	remaining: 5.72s
[09:32:01] 1099:	learn: 0.1703931	test: 0.2613759	best: 0.2611837 (1091)	total: 3.16s	remaining: 5.46s
[09:32:02] 1199:	learn: 0.1624411	test: 0.2607265	best: 0.2605512 (1189)	total: 3.47s	remaining: 5.2s
[09:32:02] 1299:	learn: 0.1554662	test: 0.2600358	best: 0.2600358 (1299)	total: 3.77s	remaining: 4.94s
[09:32:02] 1399:	learn: 0.1488305	test: 0.2595706	best: 0.2593695 (1372)	total: 4.05s	remaining: 4.63s
[09:32:03] 1499:	learn: 0.1427446	test: 0.2590065	best: 0.2585794 (1459)	total: 4.35s	remaining: 4.35s
[09:32:03] Stopped by overfitting detector  (100 iterations wait)
[09:32:03] bestTest = 0.2585794428
[09:32:03] bestIteration = 1459
[09:32:03] Shrink model to first 1460 iterations.
[09:32:03] ===== Start working with fold 8 for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost =====
[09:32:03] 0:	learn: 1.0596561	test: 1.0592903	best: 1.0592903 (0)	total: 2.62ms	remaining: 7.85s
[09:32:03] 99:	le

array([[1.3318288e-02, 9.8553562e-01, 1.1461342e-03],
       [9.8361802e-01, 1.5970919e-02, 4.1099859e-04],
       [9.8456228e-01, 1.3199883e-02, 2.2378489e-03],
       ...,
       [7.9610324e-01, 1.9727464e-01, 6.6220690e-03],
       [9.9528480e-01, 4.6610869e-03, 5.4226668e-05],
       [9.8523831e-01, 1.2616062e-02, 2.1455921e-03]], dtype=float32)

Заменим все пропуски в данных

## Обучение модели

## Оценка точности

# Cоздание файла с ответом

In [140]:
df_test = pd.read_csv("./test.csv")
print(df_test.shape)
df_test = year_of_birth_transform(df_test)
df_test = age_of_graduation_transform(df_test)

df_test = age_of_admission_transform(df_test)
df_test = organization_type_transform(df_test)

df_test = full_family_transform(df_test)
df_test = lang_transform(df_test)
df_test = normalize_grades(df_test)
#df_test = make_grouped_features(df,df_test,True)
df_submission = pd.read_csv("./submission.csv")

(6691, 23)


/home/kglin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kglin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kglin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kglin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: Set

In [141]:
probs = np.array(automl.predict(df_test).data)
preds = np.argmax(probs,axis = -1)
labels = preds.copy()

In [142]:
pd.DataFrame(probs).to_csv('probs_long-18000.csv')

In [143]:
labels[preds==0] = 4
labels[preds==1] = 3
labels[preds==2] = -1

In [144]:
df_submission['Статус'] = labels

In [145]:
df_submission

,ID,Статус
0,64996,4
1,71837,4
2,86587,3
3,73673,-1
4,54709,4
...,...,...
6686,74342,4
6687,54876,4
6688,66879,4
6689,64982,4


In [146]:
df_submission.to_csv('test_lama_pre-18000.csv',index = False)

In [131]:
probs_1 = pd.read_csv('probs_long-0.784.csv').iloc[:,-3:].values
probs_2 = pd.read_csv('probs_long-0.786054.csv').iloc[:,-3:].values
probs_3 = pd.read_csv('probs_long-0.788.csv').iloc[:,-3:].values
total_probs = (0.2*probs_1 + 0.3*probs_2+0.5*probs_3)
total_labels = np.argmax(total_probs,axis = -1)
total_labels[preds==0] = 4
total_labels[preds==1] = 3
total_labels[preds==2] = -1
df_submission['Статус'] = total_labels
df_submission.to_csv('test_lama_total_weighted.csv',index = False)

In [120]:
probs_2

,0,1,2
0,0.957538,0.030737,0.011725
1,0.983964,0.013609,0.002427
2,0.156098,0.709746,0.134157
3,0.069787,0.412607,0.517606
4,0.975138,0.023622,0.001240
...,...,...,...
6686,0.976327,0.020163,0.003510
6687,0.977122,0.021399,0.001479
6688,0.954304,0.041839,0.003857
6689,0.900306,0.092748,0.006946


In [122]:
probs_1

,0,1,2
0,0.957340,0.033805,0.008856
1,0.981870,0.015186,0.002943
2,0.153763,0.684375,0.161862
3,0.074259,0.408974,0.516767
4,0.977691,0.021493,0.000816
...,...,...,...
6686,0.975620,0.019963,0.004418
6687,0.981022,0.018020,0.000958
6688,0.957550,0.038347,0.004102
6689,0.911963,0.081593,0.006444
